In [1]:
%run d2lzh_pytorch.ipynb
import sys
import utils as d2l

import torch
import torchvision
import numpy as np


[NbConvertApp] WARNING | pattern 'file_name.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write notebook output to stdout 

ModuleNotFoundError: No module named 'utils'

In [59]:
#带入Fashion-MNIST数据集，批量大小256
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [60]:
#定义参数
#图像w h 均为28像素，则输入向量长度 28x28=784
#十个类别
num_inputs = 784
num_outputs = 10
#定义初始w权重和偏差
W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)
#确定需要学习参数
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True) 

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

# #softmax运算
在介绍如何定义softmax回归之前，我们先描述一下对如何对多维Tensor按维度操作。在下面的例子中，给定一个Tensor矩阵X。我们可以只对其中同一列（dim=0）或同一行（dim=1）的元素求和，并在结果中保留行和列这两个维度（keepdim=True）。

In [61]:
X = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(X.sum(dim=0, keepdim=True)) #列
print(X.sum(dim=1, keepdim=True)) #行


tensor([[5, 7, 9]])
tensor([[ 6],
        [15]])


在下面的函数中，矩阵X的行数是样本数，列数是输出个数。为了表达样本预测各个输出的概率，softmax运算会先通过exp函数对每个元素做指数运算，再对exp矩阵同行元素求和，最后令矩阵每行各元素与该行元素之和相除。这样一来，最终得到的矩阵每行元素和为1且非负。因此，该矩阵每行都是合法的概率分布。softmax运算的输出矩阵中的任意一行元素代表了一个样本在各个输出类别上的预测概率。

In [62]:
#sofamax 计算模块
def softmax(X):
    X_exp = X.exp()  #指数化操作
    partition = X_exp.sum(dim=1, keepdim=True)  #x（样本）指数行和
    return X_exp / partition  # 这里应用了广播机制

In [63]:
#案例
X = torch.rand((2, 5))
X_prob = softmax(X)
print(X_prob, X_prob.sum(dim=1))

tensor([[0.2049, 0.1549, 0.2458, 0.1606, 0.2338],
        [0.2457, 0.1519, 0.2009, 0.1458, 0.2557]]) tensor([1., 1.])


# 定义模型
用view将数据集展开乘num_input大小;
net:y=wx+b;
softmax(net)

In [64]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)


# 定义损失函数
使用交叉熵损失函数：
为了得到标签的预测概率，我们可以使用gather函数。在下面的例子中，变量y_hat是2个样本在3个类别的预测概率，变量y是这2个样本的标签类别。通过使用gather函数，我们得到了2个样本的标签的预测概率。与3.4节（softmax回归）数学表述中标签类别离散值从1开始逐一递增不同，在代码中，标签类别的离散值是从0开始逐一递增的。

In [65]:
#gather 解释

b = torch.Tensor([[1,2,3],[4,5,6]])
print(b)
index_1 = torch.LongTensor([[0,1],[2,0]])
index_2 = torch.LongTensor([[0,1,1],[0,0,0]])
print(torch.gather(b, dim=1, index=index_1)) #dim=1，index为行检索序号
print(torch.gather(b, dim=0, index=index_2)) #dim=0，index为列检索序号


#gather在one-hot为输出的多分类问题中，可以把最大值坐标作为index传进去，
#然后提取到每一行的正确预测结果，这也是gather可能的一个作用。

tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[1., 2.],
        [6., 4.]])
tensor([[1., 5., 6.],
        [1., 2., 3.]])


In [66]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])  #索要检索的标签序号
y_hat.gather(1, y.view(-1, 1))  #dim=1，行检索序号


def cross_entropy(y_hat, y):
    return - torch.log(y_hat.gather(1, y.view(-1, 1)))  #加负号


# 计算分类准确性
y_hat为模型预测值；y为真实类别。                                   
y_hat第一行中最大概率0.6出现在第一行第三个，对应dim=1的gather是2，而y中标签是0，错误。
第二行中国最大0.5出现在第三个，对应gather为2，于y相同。

In [67]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()
print(accuracy(y_hat, y))


0.5


# 评价模型net在数据集data_iter上的准确率。


In [73]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中描述
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n
print(evaluate_accuracy(test_iter, net))


RuntimeError: The size of tensor a (10) must match the size of tensor b (3) at non-singleton dimension 1

# 训练模型

In [71]:
num_epochs, lr = 5, 0.1

# 本函数已保存在d2lzh包中方便以后使用
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()

            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()

            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到


            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)


RuntimeError: The size of tensor a (10) must match the size of tensor b (3) at non-singleton dimension 1

# 预测
训练完成后，现在就可以演示如何对图像进行分类了。给定一系列图像（第三行图像输出），我们比较一下它们的真实标签（第一行文本输出）和模型预测结果（第二行文本输出）

In [72]:
X, y = iter(test_iter).next()

true_labels = d2l.get_fashion_mnist_labels(y.numpy())
pred_labels = d2l.get_fashion_mnist_labels(net(X).argmax(dim=1).numpy())
titles = [true + '\n' + pred for true, pred in zip(true_labels, pred_labels)]

d2l.show_fashion_mnist(X[0:9], titles[0:9])


RuntimeError: The size of tensor a (10) must match the size of tensor b (3) at non-singleton dimension 1